In [ ]:
#Instalamos la libreria de STREAMLIT
%pip install streamlit

In [ ]:
#Instalamos la libreria de PLOTLY
%pip install plotly

In [ ]:
#Instalamos librería para análisis de variables categóricas
%pip install funpymodeling

In [4]:
#Para instalar npm en visual studio
#1.Desde Google escribir node.js
#2. Instalar la versión más recomendada
! npm install localtunnel


added 22 packages in 3s

3 packages are looking for funding
  run `npm fund` for details


In [12]:
%%writefile app.py 
# Creamos el archivo de la APP en el interprete principal (Python)

#############################IMPLEMENTACIÓN DE DASHBOARD################################

# Importamos librerías
import streamlit as st
import plotly.express as px
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.metrics import r2_score, accuracy_score, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

#################################################################

# Definimos la instancia de streamlit
@st.cache_resource
def load_data():
    # Lectura del archivo csv sin índice
    df = pd.read_csv("Grecia_limpios_final (1).csv")
    return df

# Cargo los datos obtenidos de la función "load_data"
df = load_data()

# Seleccionamos las columnas numéricas del DataFrame
numeric_cols1 = df.select_dtypes(include=[np.number]).columns.tolist()

######################CREACIÓN DEL DASHBOARD################################

# 1. CREACIÓN DE LA SIDEBAR
st.sidebar.title("DASHBOARD")
st.sidebar.header("Sidebar")
st.sidebar.subheader("Panel de selección")

# 2. CREACIÓN DE LOS FRAMES
Frames = st.selectbox(label="Frames", options=["Frame 1", "Frame 2", "Frame 3", "Frame 4", "Frame 5", "Frame 6","Frame 7"])

# 3. CONTENIDO DEL FRAME 1
if Frames == "Frame 1":
    st.markdown("<h1 style='color: pink;'>Tipo de Propiedad</h1>", unsafe_allow_html=True)
    st.markdown("<h3 style='color: purple;'>Bar Plot</h3>", unsafe_allow_html=True)
    
    # Widget 2: Checkbox
    check_box = st.sidebar.checkbox(label="Mostrar Dataset")
    if check_box:
        st.write(df)
        
    # Widget 3: Selección de variable categórica
    categorical_cols = df.select_dtypes(include=['object']).columns.tolist()  # Obtener variables categóricas
    var_selected = st.sidebar.selectbox("Seleccionar variable categórica", options=categorical_cols)
    
    # Contar las frecuencias de la variable seleccionada
    freq_table = df[var_selected].value_counts().reset_index()
    freq_table.columns = [var_selected, 'Frecuencia']  # Renombrar columnas
    
    # GRAPH 1: BARPLOT
    figure1 = px.bar(
        data_frame=freq_table,
        x=var_selected,
        y='Frecuencia',
        title=f'Frecuencia de {var_selected}',
        width=1600,
        height=600,
        color_discrete_sequence=px.colors.qualitative.Dark24
    )
    
    # Mostrar el gráfico
    st.plotly_chart(figure1)

# 4. CONTENIDO DEL FRAME 2 
if Frames == "Frame 2":
    # Generamos los encabezados para el dashboard con colores
    st.markdown("<h1 style='color: pink;'>ACCOMODATES</h1>", unsafe_allow_html=True)
    st.markdown("<h3 style='color: orange;'>Scatter Plot</h3>", unsafe_allow_html=True)

    # Seleccionar columnas numéricas del DataFrame
    numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()

    # Menú desplegable de opciones de las variables numéricas graficadas
    var_num_x_selected = st.sidebar.selectbox(label="Seleccionar variable numérica para el eje X", options=numeric_cols)
    var_num_y_selected = st.sidebar.selectbox(label="Seleccionar variable numérica para el eje Y", options=numeric_cols)

    # Crear un gráfico de dispersión
    figure2 = px.scatter(data_frame=df, 
                          x=var_num_x_selected, 
                          y=var_num_y_selected, 
                          title=f"Análisis de Dispersión: {var_num_x_selected} vs {var_num_y_selected}",
                          color_discrete_sequence=['#1f77b4', '#ff7f0e'])  # Color para los puntos

    # Cambiar el color de los puntos según la variable
    figure2.for_each_trace(lambda t: t.update(marker=dict(color='#1f77b4')) if t.name == var_num_x_selected else t.update(marker=dict(color='#ff7f0e')))

    # Ajustar el diseño
    figure2.update_xaxes(title_text=var_num_x_selected)
    figure2.update_yaxes(title_text=var_num_y_selected)

    # Widget para mostrar el gráfico
    st.plotly_chart(figure2)

# 5. CONTENIDO DEL FRAME 3 (Regresión Lineal)
if Frames == "Frame 3":
    st.markdown("<h1 style='color: pink;'>Regresión Lineal</h1>", unsafe_allow_html=True)
    
    x_selected_multiple = st.sidebar.multiselect("Seleccionar variables independientes (X)", options=numeric_cols1)
    y_selected = st.sidebar.selectbox("Seleccionar variable dependiente (Y)", options=numeric_cols1)
    
    if not x_selected_multiple:
        st.error("Por favor, selecciona al menos una variable independiente.")
    else:
        # Preparar los datos
        X = df[x_selected_multiple]
        y = df[y_selected]

        # Dividir los datos en entrenamiento y prueba
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

        # Estandarización de los datos
        escalar = StandardScaler()
        X_train = escalar.fit_transform(X_train)
        X_test = escalar.transform(X_test)

        # Definir el modelo de regresión lineal
        modelo = LinearRegression()

        # Entrenar el modelo
        modelo.fit(X_train, y_train)

        # Realizar predicciones
        y_pred = modelo.predict(X_test)

        # Calcular las métricas del modelo
        r2 = r2_score(y_test, y_pred)

        # Mostrar resultados
        st.write(f"Coeficiente de determinación (R²): {r2:.2f}")

        # Mostrar coeficientes del modelo
        coef_df = pd.DataFrame({"Variable": x_selected_multiple, "Coeficiente": modelo.coef_})
        st.write(coef_df)

        # Mostrar gráfico de predicciones vs valores reales
        fig, ax = plt.subplots()
        ax.scatter(y_test, y_pred, color='red')
        ax.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'k--', lw=2)
        ax.set_xlabel('Valores Reales')
        ax.set_ylabel('Predicciones')
        st.pyplot(fig)

# 6. CONTENIDO DEL FRAME 4 (Heatmap)
if Frames == "Frame 4":
    st.markdown("<h1 style='color: pink;'>Heatmap de Correlación</h1>", unsafe_allow_html=True)
    
    # Filtrar el DataFrame para incluir solo columnas numéricas
    numeric_df = df.select_dtypes(include=[np.number])
    
    # Calcular la matriz de correlación
    correlation_matrix = numeric_df.corr()
    
    # Graficar el heatmap
    plt.figure(figsize=(10, 8))
    sns.heatmap(correlation_matrix, annot=True, fmt=".2f", cmap='coolwarm', square=True, cbar=True)
    st.pyplot(plt)

# 3. CONTENIDO DEL FRAME 5 (Regresión No Lineal)
if Frames == "Frame 5":
    st.markdown("<h1 style='color: pink;'>Regresión No Lineal</h1>", unsafe_allow_html=True)
    
    x_selected_multiple = st.sidebar.multiselect("Seleccionar variables independientes (X)", options=numeric_cols1)
    y_selected = st.sidebar.selectbox("Seleccionar variable dependiente (Y)", options=numeric_cols1)
    
    # Opciones para el tipo de regresión no lineal
    tipo_regresion = st.sidebar.selectbox("Seleccionar el tipo de regresión", options=["Polinomial", "Exponencial"])
    
    if not x_selected_multiple:
        st.error("Por favor, selecciona al menos una variable independiente.")
    else:
        # Preparar los datos
        X = df[x_selected_multiple]
        y = df[y_selected]

        # Dividir los datos en entrenamiento y prueba
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

        # Estandarización de los datos
        escalar = StandardScaler()
        X_train = escalar.fit_transform(X_train)
        X_test = escalar.transform(X_test)

        if tipo_regresion == "Polinomial":
            # Transformación polinómica
            grado = st.sidebar.slider("Seleccionar el grado del polinomio", min_value=2, max_value=5, value=2)
            poly_features = PolynomialFeatures(degree=grado)
            X_train_poly = poly_features.fit_transform(X_train)
            X_test_poly = poly_features.transform(X_test)

            # Definir el modelo de regresión lineal polinómica
            modelo = LinearRegression()

            # Entrenar el modelo
            modelo.fit(X_train_poly, y_train)

            # Realizar predicciones
            y_pred = modelo.predict(X_test_poly)

        elif tipo_regresion == "Exponencial":
            # Aplicar la transformación logarítmica a y
            y_train_log = np.log(y_train + 1)
            y_test_log = np.log(y_test + 1)

            # Definir el modelo de regresión lineal para los datos transformados
            modelo = LinearRegression()

            # Entrenar el modelo
            modelo.fit(X_train, y_train_log)

            # Realizar predicciones
            y_pred_log = modelo.predict(X_test)
            y_pred = np.exp(y_pred_log) - 1

        # Calcular las métricas del modelo
        r2 = r2_score(y_test, y_pred)

        # Mostrar resultados
        st.write(f"Coeficiente de determinación (R²): {r2:.2f}")

        # Mostrar gráfico de predicciones vs valores reales
        fig, ax = plt.subplots()
        ax.scatter(y_test, y_pred, color='blue')
        ax.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'k--', lw=2)
        ax.set_xlabel('Valores Reales')
        ax.set_ylabel('Predicciones')
        st.pyplot(fig)

# 3. CONTENIDO DEL FRAME 6 (Regresión Múltiple)
if Frames == "Frame 6":
    st.markdown("<h1 style='color: pink;'>Regresión Múltiple</h1>", unsafe_allow_html=True)

    # Selección de variables independientes y dependiente
    x_selected_multiple = st.sidebar.multiselect("Seleccionar variables independientes (X)", options=numeric_cols1)
    y_selected = st.sidebar.selectbox("Seleccionar la variable dependiente (Y)", options=numeric_cols1)
    
    if not x_selected_multiple:
        st.error("Por favor, selecciona al menos una variable independiente.")
    else:
        # Preparar los datos
        X = df[x_selected_multiple]
        y = df[y_selected]

        # Dividir los datos en entrenamiento y prueba
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

        # Estandarización de los datos
        escalar = StandardScaler()
        X_train = escalar.fit_transform(X_train)
        X_test = escalar.transform(X_test)

        # Definir el modelo de regresión lineal múltiple
        modelo = LinearRegression()

        # Entrenar el modelo
        modelo.fit(X_train, y_train)

        # Realizar predicciones
        y_pred = modelo.predict(X_test)

        # Calcular las métricas del modelo
        r2 = r2_score(y_test, y_pred)

        # Mostrar resultados
        st.write(f"Coeficiente de determinación (R²): {r2:.2f}")

        # Mostrar gráfico de predicciones vs valores reales
        fig, ax = plt.subplots()
        ax.scatter(y_test, y_pred, color='blue')
        ax.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'k--', lw=2)
        ax.set_xlabel('Valores Reales')
        ax.set_ylabel('Predicciones')
        ax.set_title('Gráfico de Predicciones vs Valores Reales')
        st.pyplot(fig)

        # Mostrar coeficientes del modelo
        coef_df = pd.DataFrame({"Variable": x_selected_multiple, "Coeficiente": modelo.coef_})
        st.write("Coeficientes del modelo:")
        st.write(coef_df)
        
# 3. CONTENIDO DEL FRAME 7 (Regresión Logística)
if Frames == "Frame 7":
    st.markdown("<h1 style='color: pink;'>Regresión Logística</h1>", unsafe_allow_html=True)
    
    # Selección de variables independientes y dependiente
    x_selected_multiple = st.sidebar.multiselect("Seleccionar variables independientes (X)", options=numeric_cols1)
    y_selected = st.sidebar.selectbox("Seleccionar la variable dependiente (Y)", options=numeric_cols1)
    
    if not x_selected_multiple:
        st.error("Por favor, selecciona al menos una variable independiente.")
    else:
        # Verificar que la variable dependiente sea binaria
        if df[y_selected].nunique() != 2:
            st.error("La variable dependiente debe ser binaria para la regresión logística.")
        else:
            # Preparar los datos
            X = df[x_selected_multiple]
            y = df[y_selected]

            # Dividir los datos en entrenamiento y prueba
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

            # Estandarización de los datos
            escalar = StandardScaler()
            X_train = escalar.fit_transform(X_train)
            X_test = escalar.transform(X_test)

            # Definir el modelo de regresión logística
            modelo = LogisticRegression()

            # Entrenar el modelo
            modelo.fit(X_train, y_train)

            # Realizar predicciones
            y_pred = modelo.predict(X_test)

            # Calcular la precisión del modelo
            accuracy = accuracy_score(y_test, y_pred)

            # Mostrar resultados
            st.write(f"Precisión del modelo: {accuracy:.2f}")

            # Mostrar la matriz de confusión
            conf_matrix = confusion_matrix(y_test, y_pred)
            fig, ax = plt.subplots()
            sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", ax=ax)
            ax.set_xlabel("Predicción")
            ax.set_ylabel("Valor Real")
            ax.set_title("Matriz de Confusión")
            st.pyplot(fig)

            # Mostrar los coeficientes del modelo
            coef_df = pd.DataFrame({"Variable": x_selected_multiple, "Coeficiente": modelo.coef_[0]})
            st.write("Coeficientes del modelo:")
            st.write(coef_df)

Overwriting app.py
